# pvanalysis

The `pvanalysis` package of `SLAM` is a tool to identify Keplerian disks in protostellar systems using position-velocity (PV) diagrams and estimate the dynamical mass of protostars if disks are present. This tool basically consists of two steps: `get_edgeridge`, which determines edge/ridge points that trace rotation curve features of PV diagrams, and `fit_edgeridge`, which performs the power-law fitting with the obtained edge/ridge points. In this note, we will briefly present how to use this tool.

In [ ]:
import numpy as np
# if you need to set a path
import sys
sys.path.append('D:\L1489_IRS_ssp\SLAM') # add PATH to SLAM

from pvanalysis import PVAnalysis
import matplotlib.pyplot as plt

In [ ]:
#%matplotlib widget

## Basic usage B6

Here is an example only with the most basic input parameters to deomnstrate the simplest usage. The first step of `pvanalysis` is to extract edge/ridge points from a PV diagram.

In [ ]:
# -------- INPUTS --------
fitsfile = "PV_diskmodels\l1489irs_i_73.0_pa_0.0_vsys_7.37_kepler_Irp_-1__with_beam_conv_upd_hdr_res_0.01_pv.fits"
outname = "PV_diskmodels\SLAMfits\l1489irs_i_73.0_pa_0.0_vsys_7.37_kepler_Irp_-1__with_beam_conv_upd_hdr_res_0.01"  # file name header for outputs
incl = 73.  # inclunation angle (deg)
vsys = 7.37  # systemic velocity (km/s)
dist = 140.  # distance to the object (pc)
rms = 6.92e-12  # rms noise level (Jy/beam)
thr = 5.  # threshold for noise cut-off for edge/ridge calculations (rms)
# -------------------------


# read PV diagram
# give rms, vsys, distance, and inclination angle
impv = PVAnalysis(fitsfile, rms, vsys, dist, incl=incl, pa=0.)
# get edge/ridge points
impv.get_edgeridge(outname, thr=thr, xlim=[-1000.,0,0,1000.], ridgemode='gauss', pixrng_vcut=3)
impv.write_edgeridge(outname=outname)

In [ ]:
# power law fitting
# --------- input parameters ----------
include_vsys = True  # vsys offset. False means vsys=0.
include_dp = False  # False means a single power
include_pin = True # False means pin=0.5 (Keplerian).
fixed_pin = 0.5  # Set the fixed pin value when include_pin is False.
fixed_dp = 0.0  # Set the fixed dp value when include_dp is False.
show_corner = True # if show corner plots or not
# -------------------------------------
impv.fit_edgeridge(include_vsys=include_vsys,
                   include_dp=include_dp,
                   include_pin=include_pin,
                   fixed_pin=fixed_pin, fixed_dp=fixed_dp,
                   outname=outname, rangelevel=0.8,
                   show_corner=show_corner)
impv.output_fitresult()


In [ ]:
# For ridge

V_b, V_b_err = 2.339, 0.000
R_b, R_b_err = 195.11 , 1.48
V_sys, V_sys_err = 7.359, 0.004
p = 0.440 
perr = 0.003



# Find scaling constant V100 (V at r = 100) using Vb, Rb... equate V_b(r/Rb)**-0.5 with V_100(r/100)**-0.5 

V_100 = V_b*np.power(R_b/100, p)

# Calculate the partial derivatives
dv_dvb = (100 / R_b)**p
dv_drb = -V_b * p * (100 / R_b)**p * (1 / R_b)
dv_dp = V_b * (100 / R_b)**p * np.log(100 / R_b)

# Calculate the propagated error using the partial derivatives
sigma_v = np.sqrt((dv_dvb * V_b_err)**2 + (dv_drb * R_b_err)**2 + (dv_dp * perr)**2)


print(round(V_100,3))
print(round(sigma_v,3))

The input parameters set free parameters and which model function (single or double power-law) is adopted. In the above case, the fitting model is a double-power law function with a fixed inner power-law index ($p_\mathrm{in}=0.5$). The fitting searches the best break point ($R_\mathrm{b}$, $V_\mathrm{b}$), where the power-law index changes, and $dp$, which is deviation of the outer power-law index from the innder one. The dynamical mass ($M_\mathrm{b}$) is estimated from the set of ($R_\mathrm{b}$, $V_\mathrm{b}$) and a given inclination angle assuming a Keplerian rotation.

The edge/ridge points and the best-fit functions can be visualized as follows.

In [ ]:
# plot results
impv.plot_fitresult(outname=outname, clevels=[-3,3,6,9],
                    vlim=[1.e-10,15.],
                    Tbcolor=True,
                    kwargs_pcolormesh={'cmap':'viridis'},
                    kwargs_contour={'colors':'lime'},                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
                    fmt={'edge':'v', 'ridge':'o'},
                    linestyle={'edge':'--', 'ridge':'-'})